In [1]:
import json
import matplotlib.pyplot as plt

import edge_validation as ev

with open('configuration/belfast.json') as f:
    config = json.load(f)

config

{'study_region': 'belfast',
 'to_crs': 'epsg:29902',
 'osm_graphml_path': 'data/Belfast/belfast_gb_2019_10000m_all_osm_20190902.graphml.xml',
 'osm_buffer_gpkg_path': 'data/Belfast/belfast_gb_2019_1600m_buffer.gpkg',
 'official_streets_shp_path': 'data/Belfast/Belfast_City_Council_Area_Street_Network.shp'}

In [2]:
%%time
# load street gdfs from osm graph and official shapefile, and the study area boundary polygon
gdf_osm_streets, gdf_official_streets, study_area = ev.load_data(config['osm_graphml_path'],
                                         config['osm_buffer_gpkg_path'],
                                         config['official_streets_shp_path'])
print(gdf_osm_streets.shape)
print(gdf_official_streets.shape)

(70312, 22)
(19757, 25)
CPU times: user 53.8 s, sys: 256 ms, total: 54 s
Wall time: 54 s


In [3]:
%%time
# clip both street gdfs to the study area boundary
gdf_osm_streets_clipped, gdf_official_streets_clipped = ev.clip_data(gdf_osm_streets, gdf_official_streets, study_area)
print(gdf_osm_streets_clipped.shape)
print(gdf_official_streets_clipped.shape)

(26244, 22)
(18662, 25)
CPU times: user 27 s, sys: 0 ns, total: 27 s
Wall time: 27 s


In [4]:
# plot map of study area + osm and official streets
fig, ax = ev.plot_data(gdf_osm_streets_clipped, gdf_official_streets_clipped, study_area)
plt.close()

In [ ]:
# Find the total length of the osm dataset edge network
ev.total_length(gdf_osm)

In [ ]:
# Find the total length of the official dataset edge network
ev.total_length(gdf_official)

In [ ]:
# Find the area of the official dataset edge network which intersects with the osm network at a specified buffer
# Change the buffer by replacing the integer in the first input
ev.buffer_intersected(5, official_buffer)

In [ ]:
# Find the area of the osm dataset edge network which intersects with the official network at a specified buffer
# Change the buffer by replacing the integer in the first input
# The buffer is measured in meters
ev.buffer_intersected(5, osm_buffer)

In [ ]:
# Find the percentage of overlap between the two networks at a specified buffer
# Change the buffer by replacing the integer in the first input
# The buffer is measured in meters
ev.shared_area(5)